In [286]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [287]:
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import utils
def trait_data(diretory):
    data=pd.read_csv(diretory, sep=";", header=None)
    data = data.rename(columns = {0:"sentences", 1:"classes"})
    x=data['sentences'].values
    y=data['classes']
    le = preprocessing.LabelEncoder()
    y=le.fit_transform(y)
    y=utils.to_categorical(y, num_classes=None, dtype="float32")
    
    return x,y

In [288]:
x_train,y_train=trait_data("../input/emotions-dataset-for-nlp/train.txt")
x_test,y_test=trait_data("../input/emotions-dataset-for-nlp/test.txt")
x_val,y_val=trait_data("../input/emotions-dataset-for-nlp/val.txt")

In [289]:
y_train

In [290]:
print('sentence: {}. Classe:{}'.format(x_train[0], y_train[0]))

In [291]:
# tokenizing

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(x_train) 
#tokenizer.fit_on_texts(x_val)
#tokenizer.fit_on_texts(x_test)
#print(x_train)
sequence_train = tokenizer.texts_to_sequences(x_train)
sequence_val = tokenizer.texts_to_sequences(x_val)
sequence_test = tokenizer.texts_to_sequences(x_test)
 
padded_train = pad_sequences(sequence_train) 
padded_test = pad_sequences(sequence_test)
padded_val = pad_sequences(sequence_val)

In [292]:
#y_train=y_train.reshape(-1,1)
#y_val=y_val.reshape(-1,1)
#y_test=y_test.reshape(-1,1)

In [293]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from tensorflow import optimizers


lr=1e-3
# build up model

model = Sequential([
                    Embedding(input_dim=5000, output_dim=16),
                    LSTM(64),
                    Dense(1000, activation='relu'),
                    Dense(200, activation='relu'),
                    Dense(50, activation='relu'),
                    Dense(25, activation='relu'),
                    Dense(6, activation='sigmoid')])

model.compile(optimizer=tf.optimizers.Adam(learning_rate=lr),loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

In [294]:
from keras.callbacks import EarlyStopping

# membuat early stopping
es = EarlyStopping(monitor = 'val_loss',
                   verbose = 0,
                   patience = 2,
                   mode = 'min',
                   restore_best_weights = True)

# training model

history = model.fit(padded_train, 
                    y_train,
                    epochs = 15,
                    validation_data = (padded_val, y_val),
                    verbose=2,
                    callbacks=[es])

In [296]:
import matplotlib.pyplot as plt

# plot loss during training
plt.subplot(121)
plt.title('Loss')
plt.plot(history.history['loss'], label='training')
plt.plot(history.history['val_loss'], label='testing')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()

# plot accuracy during training
plt.subplot(122)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='training')
plt.plot(history.history['val_accuracy'], label='testing')
plt.xlabel('Epoch')
plt.legend()
plt.show()